In [1]:
require 'rnn'
require 'itorch.Plot'

In [2]:
-- TODO something less hardcoded
filenames = {}
for i=2,9 do
    for _, typ in ipairs({'full', 'rnn'}) do
        table.insert(filenames, string.format('models_%d_%s/rubiks_best', i, typ))
        table.insert(filenames, string.format('models_%d_%s_fixed/rubiks_best', i, typ))
    end
end

best_results = {}
for _, name in ipairs(filenames) do
    best_results[name] = torch.load(name)
end

In [7]:
-- Training accuracy plot (old version with bug)
plot = itorch.Plot()
x = torch.range(2, 9)
full_y = torch.Tensor(8)
rnn_y = torch.Tensor(8)
for i=2,9 do
    name = string.format('models_%d_full/rubiks_best', i)
    full_y[i-1] = best_results[name].train_acc
    name = string.format('models_%d_rnn/rubiks_best', i)
    rnn_y[i-1] = best_results[name].train_acc
end

plot:xaxis('Episode Length'):yaxis('Training Accuracy')
plot:title('Training accuracy (50000 episodes)')
plot:line(x, full_y, 'red', 'Fully Connected')
plot:line(x, rnn_y, 'blue', 'Recurrent')
plot:legend(true)
plot:draw()

In [8]:
-- Test accuracy plot (old version with bug)
plot = itorch.Plot()
x = torch.range(2, 9)
full_y = torch.Tensor(8)
rnn_y = torch.Tensor(8)
for i=2,9 do
    name = string.format('models_%d_full/rubiks_best', i)
    full_y[i-1] = best_results[name].test_acc
    name = string.format('models_%d_rnn/rubiks_best', i)
    rnn_y[i-1] = best_results[name].test_acc
end

plot:xaxis('Episode Length'):yaxis('Test Accuracy')
plot:title('Test accuracy (50000 episodes)')
plot:line(x, full_y, 'red', 'Fully Connected')
plot:line(x, rnn_y, 'blue', 'Recurrent')
plot:legend(true)
plot:draw()

In [5]:
-- Training accuracy plot
plot = itorch.Plot()
x = torch.range(2, 9)
full_y = torch.Tensor(8)
rnn_y = torch.Tensor(8)
for i=2,9 do
    name = string.format('models_%d_full_fixed/rubiks_best', i)
    full_y[i-1] = best_results[name].train_acc
    name = string.format('models_%d_rnn_fixed/rubiks_best', i)
    rnn_y[i-1] = best_results[name].train_acc
end

plot:xaxis('Episode Length'):yaxis('Training Accuracy')
plot:title('Training accuracy (50000 episodes)')
plot:line(x, full_y, 'red', 'Fully Connected')
plot:line(x, rnn_y, 'blue', 'Recurrent')
plot:legend(true)
plot:draw()

In [6]:
-- Test accuracy plot
plot = itorch.Plot()
x = torch.range(2, 9)
full_y = torch.Tensor(8)
rnn_y = torch.Tensor(8)
for i=2,9 do
    name = string.format('models_%d_full_fixed/rubiks_best', i)
    full_y[i-1] = best_results[name].test_acc
    name = string.format('models_%d_rnn_fixed/rubiks_best', i)
    rnn_y[i-1] = best_results[name].test_acc
end

plot:xaxis('Episode Length'):yaxis('Test Accuracy')
plot:title('Test accuracy (50000 episodes)')
plot:line(x, full_y, 'red', 'Fully Connected')
plot:line(x, rnn_y, 'blue', 'Recurrent')
plot:legend(true)
plot:draw()